## Dependencies

In [ ]:
from Bio import SeqIO
import numpy as np
import pandas as pd
import gzip
import glob

## Functions

### housekeeping functions

In [ ]:
# function to count occurence of sgRNAs from raw Illumina fastq files
def count_spacers_fwd(in_lib, in_fastq, column_name):

    KEY_REGION_START = 5 # start index of key region
    KEY_REGION_END = 40 # end index of key region
    KEY = "CGAAACACCG" # identifies seq before guide to determine position

    dict_perfects = {x:0 for x in in_lib}
    try:
        handle = gzip.open(in_fastq, "rt")
    except:
        print("could not find fastq file")
        return

    readiter = SeqIO.parse(handle, "fastq")
    for record in readiter: # contains the seq and Qscore etc.
        read_sequence = str.upper(str(record.seq))
        key_region = read_sequence[KEY_REGION_START:KEY_REGION_END]
        key_index = key_region.find(KEY)
        if key_index >= 0:
            start_index = key_index + KEY_REGION_START + len(KEY)
            guide = read_sequence[start_index:(start_index + 20)]
            if guide in dict_perfects.keys(): # if sgRNA is found
                dict_perfects[guide] += 1
                
    return_df = pd.DataFrame()
    return_df['sgRNA sequence'] = dict_perfects.keys()
    return_df[column_name] = dict_perfects.values()
    return return_df

In [ ]:
# function to add columns with sgRNA counts from each condition/replicate
# df_dir is directory of tsv dataframe containing information on the sgRNAs contained in the screen
# fastq_dir_wildcard is directory containing all fastq files pertaining to samples/replicates
def add_counts_table(df_dir,fastq_dir_wildcard,df_cols):
    df = pd.read_csv(df_dir,sep='\t')
    df_ori_columns = df.columns.tolist()
    for fastq in glob.glob(fastq_dir_wildcard):
        counts_df = pd.DataFrame()
        counts_df = count_spacers_fwd(df['sgRNA sequence'],fastq,fastq.split('/')[-1].split('_')[-1].split('.')[0])
        df = pd.merge(df,counts_df,on='sgRNA sequence')
    columns_sort = df_ori_columns + df_cols
    return df[columns_sort]

### functions to generate normalized log-fold change for each sgRNA passing filter

In [ ]:
ori_columns = ['sgRNA sequence','sgRNA context sequence','Gene Symbol','Ensembl Gene ID','Ensembl transcript ID','Gene strand','Genome assembly','Transcript reference allele','Transcript alternate allele','Genome reference allele','Genome alternate allele','Chromosome','sgrna genomic position','sgRNA Strand','PAM','Edit','# edits','#silent edits','Nucleotide edits','Amino acid edits','Mutation category','Clinical significance','BsmBI flag','4T flag','Genotype_name','Residue','Mutation_type'] # specific to how sgRNAs were designed
counts_columns = ['d0','DMSO-R1','DMSO-R2','DMSO-R3','Unsorted-R1','Unsorted-R2','Unsorted-R3','Sorted-R1','Sorted-R2','Sorted-R3'] # specific to conditions in this paper

# function filtering out sgRNAs that have counts of 0 at day 0
def filter_by_d0(df_with_counts):
    return (df_with_counts[df_with_counts['d0']>0], len(df_with_counts)-len(df_with_counts[df_with_counts['d0']>0])) 

# function normalizing count by calculating reads per millions, adding pseudocount of 1 in addition to normalizing to day 0 counts
def normalize_counts(df_filtered):
    df_normalized = df_filtered.copy()
    for column in counts_columns:
        df_normalized[column+'_normalized'] = df_normalized[column].apply(lambda x: np.log2((x * 10000000 / df_normalized[column].sum()) + 1))
    for column in counts_columns[1:]: #starting from index 1 because d0 is used to normalize
        df_normalized[column+'_d0_normalized']  = df_normalized[column+'_normalized']-df_normalized['d0_normalized']
    return df_normalized

# function averaging between replicates
def merge_reps(df_normalized):
    df_reps_merged = df_normalized[ori_columns].copy()
    for i in range(1,9,3):
        columns = [x + '_d0_normalized' for x in counts_columns[i:i+3]]
        new_name = columns[0].split('_')[0].split('-')[0] + '_normalized_average'
        df_reps_merged[new_name] = df_normalized[columns].mean(axis=1)
    return(df_reps_merged)

# function normalizing to unsorted samples
def compare_to_unsorted(df_reps_merged):
    working_df = df_reps_merged.copy()
    working_df['lfc_over_unsorted'] = working_df['Sorted_normalized_average']-working_df['Unsorted_normalized_average']
    return(working_df)


### functions to calculate stats of each library

In [ ]:
# function calculating mean and standard deviation of control sgRNAs in the library
def get_plotting_params(df_compared):
    plotting_df = df_compared[['sgRNA sequence','Gene Symbol','Genotype_name','Residue','Mutation_type','lfc_over_unsorted']].copy()
    plotting_df_genic = plotting_df[~(plotting_df['Gene Symbol'].isin(['Positive_control','Negative_control'])) & ~(plotting_df['Mutation_type'].isna())]
    
    plotting_df_control = plotting_df[(plotting_df['Gene Symbol'].isin(['Negative_control']))]
    control_mean = plotting_df_control['lfc_over_unsorted'].mean()
    control_sd = plotting_df_control['lfc_over_unsorted'].std()
    
    plotting_df_genic['lfc_over_unsorted_adjusted_param1'] = plotting_df_genic['lfc_over_unsorted'] - control_mean
    
    return (plotting_df_genic,control_mean,control_sd)

## example lines of code to run analysis functions

In [ ]:
# building counts table
df_cols = ['d0','DMSO-R1','DMSO-R2','DMSO-R3','Unsorted-R1','Unsorted-R2','Unsorted-R3','Sorted-R1','Sorted-R2','Sorted-R3']

HDAC_ABE = add_counts_table("./final_files/0_231222_final_HDAC1_ABE_df.tsv","./fastq_HDAC1_ABE/HDAC1_ABE_*",df_cols)
HDAC_CBE = add_counts_table("./final_files/0_231222_final_HDAC1_CBE_df.tsv","./fastq_HDAC1_CBE/HDAC1_CBE_*",df_cols)
KBTBD4_ABE = add_counts_table("./final_files/0_231222_final_KBTBD4_ABE_df.tsv","./fastq_KBTBD4_ABE/KBTBD4_ABE_*",df_cols)
KBTBD4_CBE = add_counts_table("./final_files/0_231222_final_KBTBD4_CBE_df.tsv","./fastq_KBTBD4_CBE/KBTBD4_CBE_*",df_cols)

In [ ]:
# filtering out sgRNAs that do not have counts on day 0
HDAC_ABE_filtered, HDAC_ABE_n_excluded = filter_by_d0(HDAC_ABE)
HDAC_CBE_filtered, HDAC_CBE_n_excluded = filter_by_d0(HDAC_CBE)
KBTBD4_ABE_filtered, KBTBD4_ABE_n_excluded = filter_by_d0(KBTBD4_ABE)
KBTBD4_CBE_filtered, KBTBD4_CBE_n_excluded = filter_by_d0(KBTBD4_CBE)

In [ ]:
# normalizing counts (reads per million, pseudocount of 1, normalized over day 0)
HDAC_ABE_filtered_normalized = normalize_counts(HDAC_ABE_filtered)
HDAC_CBE_filtered_normalized = normalize_counts(HDAC_CBE_filtered)
KBTBD4_ABE_filtered_normalized = normalize_counts(KBTBD4_ABE_filtered)
KBTBD4_CBE_filtered_normalized = normalize_counts(KBTBD4_CBE_filtered)

In [ ]:
# merge reps
HDAC_ABE_averaged = merge_reps(HDAC_ABE_filtered_normalized)
HDAC_CBE_averaged = merge_reps(HDAC_CBE_filtered_normalized)
KBTBD4_ABE_averaged = merge_reps(KBTBD4_ABE_filtered_normalized)
KBTBD4_CBE_averaged = merge_reps(KBTBD4_CBE_filtered_normalized)

In [ ]:
# normalize to unsorted
HDAC_ABE_compared = compare_to_unsorted(HDAC_ABE_averaged)
HDAC_CBE_compared = compare_to_unsorted(HDAC_CBE_averaged)
KBTBD4_ABE_compared = compare_to_unsorted(KBTBD4_ABE_averaged)
KBTBD4_CBE_compared = compare_to_unsorted(KBTBD4_CBE_averaged)

In [ ]:
# calculate mean and standard deviation of control sgRNA
HDAC_ABE_plotting, HDAC_ABE_mean, HDAC_ABE_sd = get_plotting_params(HDAC_ABE_compared)
HDAC_CBE_plotting, HDAC_CBE_mean, HDAC_CBE_sd = get_plotting_params(HDAC_CBE_compared)
KBTBD4_ABE_plotting, KBTBD4_ABE_mean, KBTBD4_ABE_sd = get_plotting_params(KBTBD4_ABE_compared)
KBTBD4_CBE_plotting, KBTBD4_CBE_mean, KBTBD4_CBE_sd = get_plotting_params(KBTBD4_CBE_compared)